## 1. 数据来源及背景 ----YL----对新pyecharts做了全面认知和重新补充新代码，旧版本API不可用，pyecharts版本2.0.9
### 数据背景: 该数据集是指某公司员工的离职数据, 其包含15000个样本以及10个特征, 这10个特征分别为: 员工对公司满意度, 最新考核评估, 项目数, 月平均工作时长, 工作年限, 工作事故,离职状态, 5年内升职情况, 岗位, 薪资水平.

## 2. 明确分析目的
### 该数据集讨论的是公司员工离职的问题, 那么, 我们首先需要对影响员工离职的因素进行分析:

![离职原因等展示](1.jpg)

In [ ]:
## 我们可以将上述影响因素与现有的数据相结合来考虑问题，进而确定我们的分析目的：

## 1) 1.所分析的公司员工对公司满意度平均水平如何？
# #   2.所分析的公司员工的最新考核情况又是如何？
# #   3.所分析的公司员工所参加项目数是怎样？
# #   4.所分析的公司员工平均每月工作时长以及平均工作年限分别是多少？

## 2) 1.所分析的公司员工当前离职率是多少？
# #   2.所分析的公司员工工作事故发生率？
# #   3.所分析的公司过去5年升职率？
# #   4.所分析的公司员工薪资水平又如何？
# #   5.所分析的公司共有多少种岗位？

## 3) 公司是否离职和其他9个特征的关系如何？

## 4) 根据现有的数据, 如何对某个员工是否离职进行预测？

## 5) 针对当前的员工离职情况，企业该如何对待呢？（如何解决问题）

## 3. 数据分析

In [ ]:
# 导包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import dtype
import seaborn as sns
from pyecharts.charts import Pie,Boxplot,Bar,Grid
from pyecharts import options as opts
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn import metrics
from sklearn.metrics import classification_report
from pyecharts.globals import CurrentConfig,NotebookType
import os
from pyecharts.commons.utils import JsCode
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve
## 解决中文
plt.rcParams['font.sans-serif']=['SimHei']   # 如果是MAC，
# 不支持simhei，修改为microsoft yahei,或者arial unicode ms
plt.rcParams['axes.unicode_minus']=False
from sqlalchemy import create_engine

In [ ]:
# 利用pandas读取数据,查看数据
df=pd.read_csv(r'D:\python-learn\员工离职预测/HR_comma_sep.csv')
pd.set_option('display.max_rows', 4)
df.sort_index(inplace=True)
df

In [ ]:
## 对表中的英文名称看着观感不强，我们把英文改为中文，直观上会好很多。
df.rename(columns={"satisfaction_level":"满意度"},inplace=True)
df.rename(columns={"last_evaluation":"最新考核情况"},inplace=True)
df.rename(columns={"number_project":"参加项目数"},inplace=True)
df.rename(columns={"average_montly_hours":"月平均工作时长"},inplace=True)
df.rename(columns={"time_spend_company":"工作年限"},inplace=True)
df.rename(columns={"Work_accident":"工作事故"},inplace=True)
df.rename(columns={"left":"离职情况"},inplace=True)
df.rename(columns={"promotion_last_5years":"5年内升职情况"},inplace=True)
df.rename(columns={"sales":"岗位"},inplace=True)
df.rename(columns={"salary":"薪资等级"},inplace=True)

df

In [ ]:
# 我们可以看到该公司规模还是相当不错的，员工上万，然而，我们需要查看数据是否有丢失，那么可以查看数据类型查看是否有空
df.info()

In [ ]:
# 15000 non-null 可以看到无缺失，那么可以进行下面的分析
# 在统计数据前，首先要做一个describle 描述性概纳
df.describe()

In [ ]:
# count总数，mean平均值，std标准差，min/max 最小/最大，及四分位法
# 回到问题上来
# 针对问题 1）
## 1. 员工对公司满意度平均水平如何？
##  如descrilbe 表所示：员工对公司的满意度: 范围在0.09~1之间，跨度略大, 中位数0.640, 均值0.613, 总体来说员工对公司较满意。

## 2. 员工的最新考核情况又是如何？ last
## 最新考核评估: 可见范围是 0.36~1, 中位数0.720, 均值0.716, 员工考核平均水平显示为中等偏上。

## 3. 员工所参加项目数是怎样？
## 项目数: 范围 2~7个, 中位数4, 均值3.8, 平均参加项目数为4个

## 4.员工平均每月工作时长以及平均工作年限分别是多少？
## 平均每月工作时长: 范围96~310小时, 中位数200h, 均值201h
## 工作年限: 范围2~10年, 中位数3, 均值3.5


In [ ]:
# 针对问题 2）
## 1.当前离职率是多少？
## 当前离职率为23.81%，其中1表示离职状态，0为在职状态

## 2.工作事故发生率？
## 工作事故发生率14.46%，其中1表示有过事故发生，0为无

## 3. 过去5年升职率？
## 过去5年升职率2.13%，其中1表示有升职，0为无
## 注意：根据经验和社会现象看，此项可能为员工离职主要原因之一，后续将进行调查和分析。

df.describe(include=['O']).T  # .T 转置 ，对非数值列做describe
## 4. 薪资水平又如何？
## 从薪资水平上看，一共有三个水平，占多数的是low水平，高达7317人

## 5. 共有多少种岗位？
## 员工岗位分为十类，其中最多的岗位是sales（销售）,高达4140人

## 4.数据预处理

In [ ]:
# 赞未发现缺失值，因此不用处理缺失值
# 查看异常值：
# 通过箱线图查看异常值
fig, ax = plt.subplots(1,5, figsize=(18, 3))
sns.boxplot(x=df.columns[0], data=df, ax=ax[0])
sns.boxplot(x=df.columns[1], data=df, ax=ax[1])
sns.boxplot(x=df.columns[2], data=df, ax=ax[2])
sns.boxplot(x=df.columns[3], data=df, ax=ax[3])
sns.boxplot(x=df.columns[4], data=df, ax=ax[4])
## 可以看到除工作年限外, 其他列均无异常值. 该异常值也反映了该公司员工中可能以年轻人为主

## 5. 可视化分析

In [ ]:
 ## 在可视化分析中可以回答
## 问题 3) 公司是否离职和其他9个特征的关系如何？
## 1. 人力资源总体情况：

# 确保数据正确
print("离职情况统计:")
print(df.left.value_counts())
print(f"离职: {df.left.value_counts()[1]}")
print(f"在职: {df.left.value_counts()[0]}")

# 准备数据
attr = ["离职", "在职"]
v1 = [int(df.left.value_counts()[1]), int(df.left.value_counts()[0])]  # 确保是整数

print("绘图数据:", v1)

# 创建饼图
pie = Pie()

# 添加数据 - 使用正确的数据格式
data_pair = [list(z) for z in zip(attr, v1)]
print("数据对:", data_pair)

pie.add(
    series_name="",  # 系列名称
    data_pair=data_pair,
    radius=["35%", "65%"],  # 内外半径
    center=["50%", "50%"],  # 图表位置
)

# 设置全局配置
pie.set_global_opts(
    title_opts=opts.TitleOpts(
        title="该公司人力资源总体情况",
        pos_left="center",
        pos_top="top"
    ),
    legend_opts=opts.LegendOpts(
        orient="vertical",
        pos_left="left",
        pos_top="center"
    )
)

# 设置系列配置
pie.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position="inside",
        formatter="{b}: {c}\n({d}%)"
    )
)
pie.render("D:/python-learn/员工离职预测/pie.html")

![美观饼图](pie.jpg)

In [ ]:
# 或者为了方便，也可以给出如下matplotlib绘制饼图
labels = ['离职', '在职']
sizes = [df.left.value_counts()[1], df.left.value_counts()[0]]

# 绘制饼图
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('该公司人力资源总体情况')
plt.axis('equal')  # 保证是圆形
plt.show()

In [ ]:
df.columns=['satisfaction', 'evaluation', 'project', 'hours', 'years_work','work_accident', 'left', 'promotion', 'department', 'salary']

# 绘制箱线图
boxplot = Boxplot(init_opts=opts.InitOpts(width="800px", height="600px"))
boxplot.set_global_opts(
    title_opts=opts.TitleOpts(
        title="对公司满意度与是否离职关系图",
        pos_left="center"
    ),
    xaxis_opts=opts.AxisOpts(type_="category"),
    yaxis_opts=opts.AxisOpts(type_="value", name="满意度")
)

x_axis = ['在职', '离职']

# 将 numpy 数组转换为列表
y_axis = [
    df[df.left == 0].satisfaction.tolist(),  # 使用 tolist() 而不是 .values
    df[df.left == 1].satisfaction.tolist()
]

print("在职数据量:", len(y_axis[0]))
print("离职数据量:", len(y_axis[1]))

# 准备数据
boxplot_data = boxplot.prepare_data(y_axis)
boxplot.add_xaxis(x_axis)
boxplot.add_yaxis("满意度", boxplot_data)

boxplot.render("D:/python-learn/员工离职预测/boxplot.html")
# 就中位数而言, 离职人员对公司满意度相对较低, 且离职人员对公司满意度整体波动较大. 另外离职人员中没有满意度为1的评价.

![箱型图](boxplot.jpg)

In [ ]:
# 3. 最新考核评估与是否离职的关系
boxplot = Boxplot(init_opts=opts.InitOpts(width="800px", height="600px"))
boxplot.set_global_opts(
    title_opts=opts.TitleOpts(
        title="最新评估与是否离职关系图",
        pos_left="center"
    ),
    xaxis_opts=opts.AxisOpts(type_="category"),
    yaxis_opts=opts.AxisOpts(type_="value", name="评估分数")
)

x_axis = ['在职', '离职']

# 将 numpy 数组转换为列表
y_axis = [
    df[df.left == 0].evaluation.tolist(),  # 使用 tolist() 而不是 .values
    df[df.left == 1].evaluation.tolist()
]

# 准备数据
boxplot_data = boxplot.prepare_data(y_axis)
boxplot.add_xaxis(x_axis)
boxplot.add_yaxis("评估分数", boxplot_data)

boxplot.render("D:/python-learn/员工离职预测/boxplot2.html")

![箱型图2](boxplot2.jpg)

In [ ]:
#  4. 所参加项目与是否离职的关系
#按照项目数分组分别求离职人数和所有人数
project_left_1 = df[df.left == 1].groupby('project')['left'].count()
project_all = df.groupby('project')['left'].count()

# 分别计算离职人数和在职人数所占比例
project_left1_rate = project_left_1 / project_all
project_left0_rate = 1 - project_left1_rate
attr = project_left1_rate.index.tolist()

# 创建柱状图 - 放在左侧
bar = Bar(init_opts=opts.InitOpts(width="500px", height="400px"))
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="所参加项目数与是否离职的关系图",pos_left="10%"),
    legend_opts=opts.LegendOpts(pos_left="left", orient="vertical")
)
bar.add_xaxis(attr)
bar.add_yaxis("离职", project_left1_rate.tolist(), stack=True)
bar.add_yaxis("在职", project_left0_rate.tolist(), stack=True)

# 创建圆环图 - 放在右侧
pie = Pie(init_opts=opts.InitOpts(width="400px", height="400px"))
pie.set_global_opts(
    title_opts=opts.TitleOpts(title="各项目数所占百分比",pos_left="center"),
    legend_opts=opts.LegendOpts(orient="vertical", pos_left="center")
)
pie.add(
    '',
    [list(z) for z in zip(project_all.index.tolist(), project_all.tolist())],
    radius=["35%", "60%"],
    center=["50%", "50%"],  # 确保饼图在容器内居中
    label_opts=opts.LabelOpts(is_show=True, formatter="{b}: {c} ({d}%)")
)

# 创建网格布局 - 明确指定左右位置
grid = Grid(init_opts=opts.InitOpts(width="2000px", height="400px"))

# 柱状图放在左侧 (占据60%宽度)
grid.add(
    bar,
    grid_opts=opts.GridOpts(
        pos_left="5%",      # 左边距
        pos_right="65%",    # 右边距，为饼图留出空间
        pos_top="10%",
        pos_bottom="15%"
    )
)

# 饼图放在右侧 (占据40%宽度)
grid.add(
    pie,
    grid_opts=opts.GridOpts(
        pos_left="60%",     # 从60%位置开始
        pos_right="5%",     # 右边距
        pos_top="10%",
        pos_bottom="15%"
    )
)
grid.render("D:/python-learn/员工离职预测/grid.html")

![gird](grid.jpg)

In [ ]:
#通过上图可以发现以下2点:
# 离职率随着项目数的增多而增大, 而个别少项目数，比如2个项目数是特例，或持有其他原因,比如能力有限，选择离职
# 离职率较高的项目数6, 7在总项目数中所占百分比相对较少. 项目数为2的这部分人可能是工作能力不被认可, 其离职人数也相对较多; 项目数为6, 7的这部分人一方面体现的是工作能力较强, 另一方面也说明了工作强度大, 其可能在其他企业能有更好的发展, 自然离职率也相对较高.

In [ ]:
# 5. 平均每月工作时长和是否离职的关系
boxplot = Boxplot(init_opts=opts.InitOpts(width="800px", height="600px"))
boxplot.set_global_opts(
    title_opts=opts.TitleOpts(
        title="平均每月工作时长与是否离职关系图",
        pos_left="center"
    ),
    xaxis_opts=opts.AxisOpts(type_="category"),
    yaxis_opts=opts.AxisOpts(type_="value", name="工作时长(小时)")
)

x_axis = ['在职', '离职']

# 将 numpy 数组转换为列表
y_axis = [
    df[df.left == 0].hours.tolist(),  # 使用 tolist() 而不是 .values
    df[df.left == 1].hours.tolist()
]

# 准备数据
boxplot_data = boxplot.prepare_data(y_axis)
boxplot.add_xaxis(x_axis)
boxplot.add_yaxis("工作时长", boxplot_data)
boxplot.render("D:/python-learn/员工离职预测/boxplot3.html")

![boxplot3](boxplot3.jpg)

In [ ]:
# 通过上图可以看到:
# 离职人员的平均每月工作时长相对较长, 每月按照22个工作日计算, 结合describe(),每日工作时数的中位数为10.18小时, 最大值为14.09小时.

In [ ]:
# 6. 工作年限和是否离职的关系
# 按照工作年限分别求离职人数和所有人数
years_left_0 = df[df.left == 0].groupby('years_work')['left'].count()
years_all = df.groupby('years_work')['left'].count()

# 分别计算离职人数和在职人数所占比例
years_left0_rate = years_left_0 / years_all
years_left1_rate = 1 - years_left0_rate
attr = years_all.index.tolist()  # 转换为列表

# 创建柱状图 - 放在左侧
bar = Bar(init_opts=opts.InitOpts(width="600px", height="400px"))
bar.set_global_opts(
    title_opts=opts.TitleOpts(
        title="工作年限与是否离职的关系图",
        pos_left="10%"
    ),
    legend_opts=opts.LegendOpts(pos_left="left", orient="vertical")
)
bar.add_xaxis(attr)
bar.add_yaxis("离职", years_left1_rate.tolist(), stack=True)
bar.add_yaxis("在职", years_left0_rate.tolist(), stack=True)

# 创建圆环图 - 放在右侧
pie = Pie(init_opts=opts.InitOpts(width="400px", height="400px"))
pie.set_global_opts(
    title_opts=opts.TitleOpts(
        title="各工作年限所占百分比",
        pos_left="center"
    ),
    legend_opts=opts.LegendOpts(orient="vertical", pos_right="5%")
)
pie.add(
    '',
    [list(z) for z in zip(years_all.index.tolist(), years_all.tolist())],
    radius=["35%", "60%"],
    center=["50%", "50%"],
    label_opts=opts.LabelOpts(
        is_show=True,
        formatter="{b}: {c} ({d}%)"
    )
)

# 创建网格布局 - 明确指定左右位置
grid = Grid(init_opts=opts.InitOpts(width="1500px", height="400px"))

# 柱状图放在左侧 (占据60%宽度)
grid.add(
    bar,
    grid_opts=opts.GridOpts(
        pos_left="5%",
        pos_right="65%",
        pos_top="10%",
        pos_bottom="15%"
    )
)

# 饼图放在右侧 (占据40%宽度)
grid.add(
    pie,
    grid_opts=opts.GridOpts(
        pos_left="60%",
        pos_right="0%",
        pos_top="10%",
        pos_bottom="15%"
    )
)
grid.render("D:/python-learn/员工离职预测/grid2.html")

![grid2](grid2.jpg)

In [ ]:
#通过上图可以得出:
#在各工作年限中, 离职人员较集中于3, 4, 5, 6年, 而6年以上则相对稳定，企业中工作年限为3年的人数所占百分比最多, 其次是2年, 主要以年轻人为主

In [ ]:
# 7. 是否发生工作事故与是否离职的关系
accident_left = pd.crosstab(df.work_accident, df.left)
attr = accident_left.index.tolist()  # 转换为列表

bar = Bar(init_opts=opts.InitOpts(width="1000px", height="600px"))
bar.set_global_opts(
    title_opts=opts.TitleOpts(
        title="是否发生工作事故与是否离职的关系图",
        pos_left="center"
    ),
    legend_opts=opts.LegendOpts(
        pos_left="left",
        orient="vertical"
    )
)
bar.add_xaxis(attr)
bar.add_yaxis(
    "离职",
    accident_left[1].tolist(),  # 转换为列表
    stack=True
)
bar.add_yaxis(
    "在职",
    accident_left[0].tolist(),  # 转换为列表
    stack=True
)
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True)
)
bar.render("D:/python-learn/员工离职预测/bar.html")

![bar](bar.jpg)

In [ ]:
## 可以看到少部分出现工作事故, 且其中有较少部分人离职.
## 说明工作事故与离职之间的关系比较模糊，暂不能盖棺定论

In [ ]:
# 8. 5年内是否升职与是否离职的关系
promotion_left = pd.crosstab(df.promotion, df.left)
attr = promotion_left.index.tolist()  # 转换为列表

bar = Bar(init_opts=opts.InitOpts(width="800px", height="600px"))
bar.set_global_opts(
    title_opts=opts.TitleOpts(
        title="5年内是否升职与是否离职的关系图",
        pos_left="center"
    ),
    legend_opts=opts.LegendOpts(
        pos_left="left",
        orient="vertical"
    )
)
bar.add_xaxis(attr)
bar.add_yaxis(
    "离职",
    promotion_left[1].tolist(),  # 转换为列表
    stack=True
)
bar.add_yaxis(
    "在职",
    promotion_left[0].tolist(),  # 转换为列表
    stack=True
)
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True)
)
bar.render("D:/python-learn/员工离职预测/bar2.html")

![bar2](bar2.jpg)

In [ ]:
# 可以看到5年内多数人没有升职, 离职率就相对较高.
# 可能说明，晋升路狭窄或晋升难，这将导致离职率上升。


In [ ]:
# 9. 岗位与是否离职的关系
# 分别计算各岗位离职人员比例和各岗位占总体百分比
department_left_0 = df[df.left == 0].groupby('department')['left'].count()
department_all = df.groupby('department')['left'].count()
department_left0_rate = department_left_0 / department_all
department_left1_rate = 1 - department_left0_rate
attr = department_all.index.tolist()  # 转换为列表

# 创建柱状图
bar = Bar(init_opts=opts.InitOpts(width="1200px", height="350px"))
bar.set_global_opts(
    title_opts=opts.TitleOpts(
        title="岗位与离职比例的关系图",
        pos_top="40%"
    ),
    legend_opts=opts.LegendOpts(
        pos_top="45%",
        pos_right="80%"
    ),
    xaxis_opts=opts.AxisOpts(
        interval=0,
        axislabel_opts=opts.LabelOpts(rotate=30)
    ),
    datazoom_opts=opts.DataZoomOpts(is_show=True)
)
bar.add_xaxis(attr)
bar.add_yaxis("离职", department_left1_rate.tolist(), stack=True)
bar.add_yaxis("在职", department_left0_rate.tolist(), stack=True)

# 创建圆环图
pie = Pie(init_opts=opts.InitOpts(width="1200px", height="350px"))
pie.set_global_opts(
    title_opts=opts.TitleOpts(
        title="各个岗位所占百分比",
        pos_left="left"
    ),
    legend_opts=opts.LegendOpts(
        orient="vertical",
        pos_right="80%",
        pos_top="4%"
    )
)
pie.add(
    '',
    [list(z) for z in zip(department_all.index.tolist(), department_all.tolist())],
    center=["50%", "23%"],
    radius=["18%", "35%"],
    label_opts=opts.LabelOpts(
        is_show=True,
        formatter="{b}: {c} ({d}%)"
    )
)

# 创建网格布局
grid = Grid(init_opts=opts.InitOpts(width="1200px", height="700px"))
grid.add(
    bar,
    grid_opts=opts.GridOpts(
        pos_top="50%",
        pos_bottom="25%"
    )
)
grid.add(
    pie,
    grid_opts=opts.GridOpts(
        pos_top="5%",
        pos_bottom="55%"
    )
)
grid.render("D:/python-learn/员工离职预测/grid3.html")

![grid](grid3.jpg)

In [ ]:
# 通过下图可以看出:
# 在所有岗位中销售岗位人数所占百分比最多, 达到27.6%, 最少是管理层, 其所占百分比是4.2%
# 令人意外的是岗位中hr离职率最高

In [ ]:
# 10. 薪资水平和是否离职的关系
#按照薪资水平分别求离职人数和所有人数
salary_left = pd.crosstab(df.salary, df.left).sort_values(0, ascending = False)
attr = salary_left.index.tolist()  # 转换为列表

bar = Bar(init_opts=opts.InitOpts(width="800px", height="600px"))
bar.set_global_opts(
    title_opts=opts.TitleOpts(
        title="薪资水平和是否离职的关系图",
        pos_left="center"
    ),
    legend_opts=opts.LegendOpts(
        pos_left="left",
        orient="vertical"
    )
)
bar.add_xaxis(attr)
bar.add_yaxis(
    "离职",
    salary_left[1].tolist(),  # 转换为列表
    stack=True
)
bar.add_yaxis(
    "在职",
    salary_left[0].tolist(),  # 转换为列表
    stack=True
)
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True)
)
bar.render("D:/python-learn/员工离职预测/bar3.html")

![bar3](bar3.jpg)

In [ ]:
# 薪资分为三个水平: low, medium, high. low水平离职人数最多, 所占比例也最大, 而high则最少. 但光从数据上看并不能说明什么，应当对照比例看，然而比例上得到的也和结论相似，即工资水平低的离职率会高一些。

## 6. 特征工程

In [ ]:
# 1. 离散型数据处理 ------离散型数据可分为两种: 一种是定序, 一种是定类.
## 1) 定序
## 薪资水平其含有顺序意义, 因此将其字符型转化为数值型
df['salary'] = df.salary.map({
            "low": 0, "medium": 1, "high": 2})
df.salary.unique()

In [ ]:
## 2) 定类
## 岗位是定类型变量, 对其进行one-hot编码, 这里直接利用pandas的get_dummies方法.
df_one_hot = pd.get_dummies(df, prefix="dep")
df_one_hot.shape
print(df_one_hot.columns.tolist())

In [ ]:
## 2. 连续型数据处理
## 逻辑回归模型能够适应连续型变量, 因此可以不用进行离散化处理, 又由于多个特征之间差异差异较大会造成梯度下降算法收敛速度变慢, 故进行归一化处理
## 采用max-min归一化方法
hours = df_one_hot['average_montly_hours']
df_one_hot['average_montly_hours'] = df_one_hot.average_montly_hours.apply(lambda x: (x-hours.min()) / (hours.max()-hours.min()))

In [ ]:
## 3. 相关系数
## 两个变量均是连续型且具有线性关系, 则可以使用皮尔逊相关系数, 否则使用斯皮尔曼相关系数, 这里采用斯皮尔曼相关系数
#计算相关系数
correlation = df_one_hot.corr(method = "spearman")
plt.figure(figsize=(18, 10))
#绘制热力图
sns.heatmap(correlation, linewidths=0.2, vmax=1, vmin=-1, linecolor='w',fmt='.2f',
            annot=True,annot_kws={
            'size':10},square=True)

## 7. 逻辑回归模型

In [ ]:
##  1. 划分数据集
#划分训练集和测试集
X = df_one_hot.drop(['left'], axis=1)
y = df_one_hot['left']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
## 2. 训练模型
LR = LogisticRegression()
print(LR.fit(X_train, y_train))
print("训练集准确率: ", LR.score(X_train, y_train))
print("测试集准确率: ", LR.score(X_test, y_test))

In [ ]:
## 参考官方文档说明, 参数C是正则化项参数的倒数, C的数值越小, 惩罚的力度越大. penalty可选L1, L2正则化项, 默认是L2正则化.
# 参数solver可选{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}
# newton-cg, lbfgs是拟牛顿法,
# liblinear是坐标轴下降法,
# sag, saga是随机梯度下降法, saga可以适用于L1和L2正则化项, 而sag只能用于L2正则化项.
## 指定随机梯度下降优化算法选择saga，看看能否提升准确率
LR = LogisticRegression(solver='saga')
print(LR.fit(X_train, y_train))
print("训练集准确率: ", LR.score(X_train, y_train))
print("测试集准确率: ", LR.score(X_test, y_test))
## 在选择随机梯度下降法后, 训练集和测试集准确率均略有提升.但幅度不大

In [ ]:
## 3. 调参
## 用准确率进行10折交叉验证选择合适的参数C
Cs = 10**np.linspace(-10, 10, 500)
lr_cv = LogisticRegressionCV(Cs=Cs, cv=10, penalty='l2', solver='saga',  max_iter=10000, scoring='accuracy')
lr_cv.fit(X_train, y_train)
lr_cv.C_

In [ ]:
## 用求到的C预测
LR = LogisticRegression(solver='saga', penalty='l2', C=2.2188534)
print(LR.fit(X_train, y_train))
print("训练集准确率: ", LR.score(X_train, y_train))
print("测试集准确率: ", LR.score(X_test, y_test))

In [ ]:
## 多次尝试求解后，提升甚微，可以多次改参尝试。对于二分类问题, 准确率有时不是很好的评估方法, 这时需要用到混淆矩阵
X_train_pred = LR.predict(X_train)
X_test_pred = LR.predict(X_test)
print('训练集混淆矩阵:')
print(metrics.confusion_matrix(y_train, X_train_pred))
print('测试集混淆矩阵:')
print(metrics.confusion_matrix(y_test, X_test_pred))

In [ ]:
## 打印训练集和测试集
print('逻辑训练集:')
print(classification_report(y_train, X_train_pred))
print('逻辑测试集:')
print(classification_report(y_test, X_test_pred))

In [ ]:
## 在训练集有0.83的精准率和0.93的召回率, 在测试集上有0.83的精准率和0.92的召回率.

## 8. 朴素贝叶斯模型

In [ ]:
#构建高斯朴素贝叶斯模型
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print("训练集准确率: ", gnb.score(X_train, y_train))
print("测试集准确率: ", gnb.score(X_test, y_test))
X_train_pred =gnb.predict(X_train)
X_test_pred = gnb.predict(X_test)
print('训练集混淆矩阵:')
print(metrics.confusion_matrix(y_train, X_train_pred))
print('测试集混淆矩阵:')
print(metrics.confusion_matrix(y_test, X_test_pred))
print('训练集:')
print(classification_report(y_train, X_train_pred))
print('测试集:')
print(classification_report(y_test, X_test_pred))

In [ ]:
## 可以看到其召回率较逻辑回归低, 但是精准率高于逻辑回归.
## 逻辑训练：0.83 0.93        朴素贝叶斯训练：0.91 0.73
## 逻辑测试：0.83 0.92        朴素贝叶斯测试：0.92 0.72

## 9. 模型评估之ROC曲线

In [ ]:
#将逻辑回归模型和高斯朴素贝叶斯模型预测出的概率均与实际值通过roc_curve比较返回假正率, 真正率, 阈值
lr_fpr, lr_tpr, lr_thresholds = roc_curve(y_test, LR.predict_proba(X_test)[:,1])
gnb_fpr, gnb_tpr, gnb_thresholds = roc_curve(y_test, gnb.predict_proba(X_test)[:,1])
#分别计算这两个模型的auc的值, auc值就是roc曲线下的面积
lr_roc_auc = metrics.auc(lr_fpr, lr_tpr)
gnb_roc_auc = metrics.auc(gnb_fpr, gnb_tpr)
plt.figure(figsize=(8, 5))
plt.plot([0, 1], [0, 1],'--', color='r')
plt.plot(lr_fpr, lr_tpr, label='LogisticRegression(area = %0.2f)' % lr_roc_auc)
plt.plot(gnb_fpr, gnb_tpr, label='GaussianNB(area = %0.2f)' % gnb_roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.show()

In [ ]:
## ROC曲线越靠近左上角说明分类效果越好, 与之对应的auc的值就越大. 仅对于该数据集来说, 高斯朴素贝叶斯模型略优于逻辑回归模型。（可能不全面，后续应该再讨论）

In [ ]:
## 5) 针对当前的员工离职情况，企业该如何对待呢？
# 该公司的员工离职现状:
# 1.当前离职率为23.81%
# 2.离职人员对公司满意度普遍较低
# 3.离职人员的考核成绩相对较高, 说明离职人员多数为优秀人才.
# 4.项目数范围为2~7个, 其中参加7个项目的离职率最高，其次是2个的; 7个的工作能力较强, 在其他企业有更好的发展, 2个的可能是在该公司中工作能力不被认可
# 5.离职人员的平均每月工作时长较长
# 6.离职人员的工作年限集中在3到6年
# 7.5年内未升职的离职率较高
# 8.hr岗位的离职率最高, 目前企业普遍存在"留人难, 招人难”，这可能是导致该岗位的离职率高的主要原因
# 9.低等薪资水平的离职率最高

# 由于企业培养人才是需要大量的成本, 为了防止人才再次流失, 因此应当注重解决人才的流失问题, 也就是留人, 另外如果在招人时注意某些问题, 也能在一定程度上减少人才流失. 因此, 这里可将对策分为两种, 一种是留人对策, 一种是招人对策.

# 留人对策:                                     招人对策:
# 1.建立良好的薪酬制度, 不得低于市场水平             1.明确企业招聘需求, 员工的能力应当与岗位需求相匹配
# 2.建立明朗的晋升机制                            2.与应聘者坦诚相见
# 3.完善奖惩机制, 能者多劳, 也应多得.               3.招聘期间给予的相关承诺必须实现
# 4.实现福利多样化, 增加员工对企业的忠诚度            4.欢迎优秀流失人才回归
# 5.重视企业文化建设, 树立共同的价值观
# 6.改善办公环境以及营造良好的工作氛围
# 7.鼓励员工自我提升

In [ ]:
## 本文仅从逻辑回归和朴素贝叶斯模型上入手，从结果上看，表现只能说一般，由于特征工程处理不佳，在实际应用中，可选择主成分分析对更优选择的特征进行选择